# Analyses of the Hofree et al. original datasets

## Import all the preprocessed data in Matlab format
For the current analyses, all the required preprocessed data are already in the folder "data."

If you want to generate them from the original Hofree et al. dataset, you have to use the script "Matlab2Python.m" in the "tools" folder. The original dataset from Hofree et al. with their Matlab code is available on the UCSD's [Network Based Stratification](http://chianti.ucsd.edu/~mhofree/wordpress/?page_id=26) webpage.

In [0]:
from scipy.io import loadmat
dataFolder='data/'

# Patients' somatic mutation profiles
somatic = loadmat(dataFolder+'somatic_data_UCEC.mat')
samples_id = [k[0][0][:12] for k in somatic['sample_id']]
# Patients' full phenotypes
phenotypes = loadmat(dataFolder+'UCEC_clinical_phenotype.mat')
patients = [c[0][0] for c in phenotypes['UCECppheno'][0][0][0]]
tmp = [c[0][0] for c in phenotypes['UCECppheno'][0][0][10]]
cancer = [tmp[patients.index(p)] for p in samples_id]
tmp = [c[0][0] for c in phenotypes['UCECppheno'][0][0][17]]
grade = [tmp[patients.index(p)] for p in samples_id]

# Adjacency matrix
network = loadmat(dataFolder+'adj_mat.mat')
# Correspondance between matrices rows number and entrez id
entrez_to_idmat = loadmat(dataFolder+'entrez_to_idmat.mat')

## Check preprocessed data format

In [0]:
print somatic.keys()
len(somatic['gene_id_all'])

In [0]:
mutations=somatic['gene_indiv_mat']
mutations.shape

In [0]:
print network.keys()
net=network['adj_mat']
net.shape

In [0]:
entrez_to_idmat.keys()

In [0]:
len(entrez_to_idmat['keymat'][0])

## Extract all the ids

In [0]:
keys=[x[0] for x in entrez_to_idmat['keymat'][0]]
ids=[x[0][0] for x in entrez_to_idmat['entrezid'][0]]
genes = [x[0] for x in somatic['gene_id_all']]

In [0]:
print "Ensembl ID:", keys[0]
print "Entrez ID:", ids[0]
print "Check on NCBI: http://www.ncbi.nlm.nih.gov/gene/%i" % ids[0]

## Extract indexes of the genes in the adjacency matrix

In [0]:
import numpy as np
l=[]
subnet=[]
good=[]
bad=[]
for j,g in enumerate(genes):
    try:
        i=ids.index(g)
        subnet.append(i)
        good.append(j)
    except:
        i=np.nan
        bad.append(j)
    l.append(i)
    
subnetNotmutated=[g for g in range(net.shape[1]) if not(g in subnet)]

In [0]:
print "All mutated genes:",len(l)
print "Referenced in the PPI:",len(good)
print "On their own:",len(bad)
print "In the PPI but not mutated:",len(subnetNotmutated)

## Extract the submatrices of references genes & zero-padding of the adjacency matrix

In [0]:
AA=np.array(net[subnet][:,subnet].todense())
AA=AA-np.diag(np.diag(AA))
AB=np.array(net[subnet][:,subnetNotmutated].todense())
AC=np.zeros([AA.shape[0],len(bad)])
BA=np.array(net[subnetNotmutated][:,subnet].todense())
BB=np.array(net[subnetNotmutated][:,subnetNotmutated].todense())
BB=BB-np.diag(np.diag(BB))
BC=np.zeros([BB.shape[0],len(bad)])
CA=np.zeros([len(bad),AA.shape[0]])
CB=np.zeros([len(bad),BB.shape[0]])
CC=np.diagflat(np.zeros(len(bad)))
print AA.shape, BB.shape, CC.shape

In [0]:
nnnet=np.bmat([[AA,AB,AC],[BA,BB,BC],[CA,CB,CC]])
nnmut=np.bmat([mutations[:,good],np.zeros([mutations.shape[0],BB.shape[0]]),mutations[:,bad]])
symbols=somatic['gene_id_symbol'][good+bad]

In [0]:
print "Network size:",nnnet.shape
print "Mutation size:",nnmut.shape

In [0]:
%matplotlib inline
import matplotlib.pyplot as plt
degree=np.squeeze(np.array(nnnet.sum(axis=0)))
plt.figure(1,figsize=(16,10))
plt.plot(degree)
plt.ylabel("Degree (number of neighboors in the PPI)")
plt.xlabel("Genes (keys)")
plt.show()

## Filtering according to Hofree et al.

### Computing network influence score [warning: very long!]
For more details, see: 

Vandin, F., Upfal, E., & Raphael, B. J. (2011). Algorithms for Detecting Significantly Mutated Pathways in Cancer. Journal of Computational Biology, 18(3), 507–522. http://doi.org/10.1089/cmb.2010.0265

Vanunu O, Magger O, Ruppin E, Shlomi T, Sharan R (2010) Associating Genes and Protein Complexes with Disease via Network Propagation. PLoS Comput Biol 6(1): e1000641. doi:10.1371/journal.pcbi.1000641



In [0]:
#remove genes on their own
nnnetFiltered=nnnet[degree>0,:][:,degree>0]
nnnetFiltered.shape

In [0]:
from numpy import linalg as LA
from scipy.io import savemat
diffusionFactor=0.7
computeInfluence=False
if computeInfluence:
    from IPython.html.widgets import FloatProgress
    from IPython.display import display 

    influence=np.zeros(nnnetFiltered.shape)
    influencers=np.zeros(nnnetFiltered.shape)
    f = FloatProgress(min=0, max=nnnetFiltered.shape[0])
    display(f)
    
    PPIAdjacencyMatrix=nnnetFiltered+np.diagflat(np.ones(nnnetFiltered.shape[0]))
    tmp=np.array(PPIAdjacencyMatrix.sum(axis=0))
    tmp=np.diagflat(1./tmp)**1/2
    tmp2=np.dot(tmp,PPIAdjacencyMatrix)
    A=np.dot(tmp2,tmp)
    
    for i in range(nnnetFiltered.shape[0]):
        f.value = i
        mutationProfile=np.zeros(nnnetFiltered.shape[0])
        mutationProfile[i]=1
        X1=mutationProfile
        X2=diffusionFactor*X1*A+(1-diffusionFactor)*mutationProfile
        while LA.norm(X2-X1)>10e-6:
            X1=X2
            X2=diffusionFactor*X1*A+(1-diffusionFactor)*mutationProfile
        influence[i,:]=np.squeeze(X2)
    
    #Save the raw influence distance matrix (heavy!)
    savemat(dataFolder+'influenceDistance.mat',{'influence':influence, 'diffusionFactor':diffusionFactor})
    #shasum(1) = 184a64289651ec1034e47cce26bb332c64333166
    #md5 = 927371b04488b48878ca597122eae8e9
    #Save the sparse influence distance by merging with PPI
    from scipy.sparse import lil_matrix
    PPI_influence=lil_matrix(np.multiply(np.min(np.dstack((influence, influence.T)),axis=2),np.array(nnnetFiltered)))
    savemat(dataFolder+'PPI_influence.mat',{'PPI_influence':PPI_influence, 'diffusionFactor':diffusionFactor}, do_compression=True)
else:
    influence_data = loadmat(dataFolder+'PPI_influence.mat')
    PPI_influence=influence_data['PPI_influence']
    diffusionFactor=influence_data['diffusionFactor'][0][0]

In [0]:
plt.figure(figsize=(16,16))
plt.spy(PPI_influence, markersize=1)
plt.show()

## Keeping only the connections with the best influencers
"The degree to which local network topology versus global network topology constrains W is determined by the number of nearest neighbors. We experimented with neighbor counts ranging from 5 to 50 to include in the nearest network, and we observed only small changes in outcome (data not shown). For the work presented in this manuscript, the 11 most influential neighbors of each gene in the network as determined by network influence distance were used."

In [0]:
PPIneighboorsMax=11
influenceMat=PPI_influence.todense()
newnet=np.zeros(nnnetFiltered.shape)
for i in range(nnnetFiltered.shape[0]):
    bestInfluencers=np.argsort(influenceMat[i,:])[:,-PPIneighboorsMax:]
    newnet[i,bestInfluencers]=np.squeeze(np.array(nnnetFiltered[i,bestInfluencers]))
newnet=np.max(np.dstack((newnet, newnet.T)),axis=2)

In [0]:
plt.figure(1,figsize=(18,9))
plt.subplot(121)
plt.imshow(nnnetFiltered)
plt.set_cmap('Greys')
plt.title("Original adjacency")
plt.subplot(122)
plt.imshow(newnet)
plt.title("With only the "+str(PPIneighboorsMax)+" best influencers")
plt.show()

In [0]:
plt.figure(1,figsize=(16,10))
plt.plot(newnet.sum(axis=0))
plt.show()

In [0]:
print nnnet.shape, nnnetFiltered.shape, nnnet[degree>0,:][:,degree>0].shape, newnet.shape

In [0]:
keepSingletons=False
mutationsMin=10

In [0]:
nnnetFiltered=nnnet[degree>0,:][:,degree>0]
filteredGenes=degree==0
filteredGenes[filteredGenes==False]=newnet.sum(axis=1)==0
print "%i genes without neighboor after filtering with maximum %i influencers criterion" % (filteredGenes.sum(), PPIneighboorsMax)

In [0]:
if keepSingletons:
    netFinal=np.bmat([[np.matrix(newnet), np.matrix(np.zeros([newnet.shape[0],sum(degree==0)]))], [np.matrix(np.zeros([sum(degree==0),newnet.shape[0]])), np.matrix(np.diagflat(np.zeros(sum(degree==0))))]])
    mutFinal=np.concatenate([nnmut[:,filteredGenes==False],nnmut[:,filteredGenes==True]], axis=1)
else:
    netFinal=newnet
    mutFinal=nnmut[:,filteredGenes==False]

In [0]:
filteredPatients=np.ndarray.flatten(np.array(mutFinal.sum(axis=1)))<mutationsMin
mutFinal=mutFinal[filteredPatients==False,:]

print "Removing %i patients with less than %i mutations" % (filteredPatients.sum(),mutationsMin)
print "New adjacency matrix:",netFinal.shape
print "New mutation profile matrix:",mutFinal.shape

In [0]:
plt.figure(1,figsize=(16,10))
plt.plot(np.array(netFinal).sum(axis=0))
plt.show()

## Diffusion of the mutation profiles according to the PPI

In [0]:
import scipy.sparse as sp
norm = lambda x: np.sqrt(x.multiply(x).sum())
def mutationProfileDiffusion(mutationProfile,PPIAdjacencyMatrix,diffusionFactor):
    n = PPIAdjacencyMatrix.shape[0]
    PPIAdjacencyMatrix=PPIAdjacencyMatrix+sp.dia_matrix((np.ones(PPIAdjacencyMatrix.shape[0]), [0]), shape=(n, n))
    
    sums = 1.0 / PPIAdjacencyMatrix.sum(axis=0)
    d = sp.dia_matrix((sums, [0]), shape=(n, n))
    A = PPIAdjacencyMatrix.dot(d)
    
    X1=mutationProfile
    X2=diffusionFactor*X1.dot(A)+(1-diffusionFactor)*mutationProfile
    while norm(X2-X1)>10e-6:
        X1=X2
        X2=diffusionFactor*X1.dot(A)+(1-diffusionFactor)*mutationProfile
    return X2

In [0]:
mutDiffused=mutationProfileDiffusion(sp.csr_matrix(mutFinal),sp.csr_matrix(netFinal), diffusionFactor)
mutDiffused[np.isnan(mutDiffused.todense())]=0
mutDiffused=mutDiffused.todense()

In [0]:
def quantile_normalization(anarray):
        A=anarray.T
        AA = np.zeros_like(A)
        I = np.argsort(A,axis=0)
        AA[I,np.arange(A.shape[1])] = np.mean(A[I,np.arange(A.shape[1])],axis=1)[:,np.newaxis]
        return AA.T

In [0]:
mutQDiffused=quantile_normalization(np.squeeze(np.asarray(mutDiffused)))

In [0]:
plt.figure(1,figsize=(16,10))
plt.subplot(311)
plt.plot(np.squeeze(np.array(mutFinal[0,:])))
plt.xlim([0,mutFinal.shape[1]])
plt.title("Original mutation profile")
plt.subplot(312)
plt.plot(np.squeeze(np.array(mutDiffused[0,:])))
plt.xlim([0,mutFinal.shape[1]])
plt.title("Diffused mutation profile")
plt.subplot(313)
plt.plot(mutQDiffused[0,:])
plt.xlim([0,mutFinal.shape[1]])
plt.title("Quantile Normalized Diffused mutation profile")
plt.show()

In [0]:
plt.figure(1,figsize=(16,5))
plt.subplot(411)
plt.imshow(mutFinal)
plt.title("Original mutation profile")
plt.subplot(412)
plt.imshow(mutDiffused)
plt.title("Diffused mutation profile")
plt.subplot(413)
plt.imshow(mutQDiffused)
plt.title("Quantile Normalized Diffused mutation profile")
plt.subplot(414)
plt.hist(np.array(np.squeeze(mutQDiffused.reshape((1,-1)))).T, 50, normed=1, histtype='stepfilled')
plt.title("Weigths histogram after diffusion")
plt.show()

## Non-Negative Matrix (NMF) decomposition 

In [0]:
from sklearn.decomposition import ProjectedGradientNMF
model = ProjectedGradientNMF(n_components=3, init='nndsvdar', random_state=0)
model.fit(np.matrix(mutFinal))
sklearnComp=model.components_.copy()
sklearnStrat=np.argmax(model.transform(np.matrix(mutFinal)),axis=1).copy()
model.fit(np.matrix(mutDiffused))
sklearnCompDiff=model.components_.copy()
sklearnStratDiff=np.argmax(model.transform(np.matrix(mutDiffused)),axis=1).copy()
model.fit(np.matrix(mutQDiffused))
sklearnCompQDiff=model.components_.copy()
sklearnStratQDiff=np.argmax(model.transform(np.matrix(mutQDiffused)),axis=1).copy()

plt.figure(1,figsize=(16,10))
plt.subplot(411)
plt.plot(sklearnComp.T/sklearnComp.max())
plt.ylabel("Weight")
plt.xlabel("Genes")
plt.title("NMF decomposition on raw mutation profiles")
plt.xlim([0,sklearnComp.shape[1]])
plt.subplot(412)
plt.plot(sklearnCompDiff.T/sklearnCompDiff.max())
plt.ylabel("Weight")
plt.xlabel("Genes")
plt.title("NMF decomposition on diffused mutation profiles")
plt.xlim([0,sklearnCompDiff.shape[1]])
plt.subplot(413)
plt.plot(sklearnCompQDiff.T/sklearnCompQDiff.max())
plt.ylabel("Weight")
plt.xlabel("Genes")
plt.title("NMF decomposition on quantile normlaized diffused mutation profiles")
plt.xlim([0,sklearnCompDiff.shape[1]])
plt.subplot(414)
plt.plot(sklearnCompQDiff.T/sklearnCompQDiff.max()-sklearnCompDiff.T/sklearnCompDiff.max())
plt.ylabel("Weight difference")
plt.xlabel("Genes")
plt.title("Difference")
plt.xlim([0,sklearnCompDiff.shape[1]])
plt.legend({'Component 1','Component 2','Component 3'})
plt.show()

## GNMF Implementation

In [0]:
## Reuse scikit-learn functions
from sklearn.utils import check_random_state
from sklearn.utils.extmath import randomized_svd, safe_sparse_dot

def check_non_negative(X, whom):
    X = X.data if sp.issparse(X) else X
    if (X < 0).any():
        raise ValueError("Negative values in data passed to %s" % whom)

def _sparseness(x):
    """Hoyer's measure of sparsity for a vector"""
    sqrt_n = np.sqrt(len(x))
    return (sqrt_n - LA.norm(x, 1) / LA.norm(x)) / (sqrt_n - 1)

def safe_vstack(Xs):
    if any(sp.issparse(X) for X in Xs):
        return sp.vstack(Xs)
    else:
        return np.vstack(Xs)

def NBS_init(X,n_components,init=None):
        n_samples, n_features = X.shape
        if init is None:
            if n_components < n_features:
                init = 'nndsvd'
            else:
                init = 'random'


        if init == 'nndsvd':
            W, H = _initialize_nmf(X, n_components)
        elif init == "random":
            rng = check_random_state(random_state)
            W = rng.randn(n_samples, n_components)
            # we do not write np.abs(W, out=W) to stay compatible with
            # numpy 1.5 and earlier where the 'out' keyword is not
            # supported as a kwarg on ufuncs
            np.abs(W, W)
            H = rng.randn(n_components, n_features)
            np.abs(H, H)
        else:
            raise ValueError(
                'Invalid init parameter: got %r instead of one of %r' %
                (init, (None, 'nndsvd', 'nndsvda', 'nndsvdar', 'random')))
        return W, H

def _initialize_nmf(X, n_components, variant=None, eps=1e-6,
                    random_state=None):
    """NNDSVD algorithm for NMF initialization.

    Computes a good initial guess for the non-negative
    rank k matrix approximation for X: X = WH

    Parameters
    ----------

    X : array, [n_samples, n_features]
        The data matrix to be decomposed.

    n_components : array, [n_components, n_features]
        The number of components desired in the approximation.

    variant : None | 'a' | 'ar'
        The variant of the NNDSVD algorithm.
        Accepts None, 'a', 'ar'
        None: leaves the zero entries as zero
        'a': Fills the zero entries with the average of X
        'ar': Fills the zero entries with standard normal random variates.
        Default: None

    eps: float
        Truncate all values less then this in output to zero.

    random_state : numpy.RandomState | int, optional
        The generator used to fill in the zeros, when using variant='ar'
        Default: numpy.random

    Returns
    -------

    (W, H) :
        Initial guesses for solving X ~= WH such that
        the number of columns in W is n_components.

    Remarks
    -------

    This implements the algorithm described in
    C. Boutsidis, E. Gallopoulos: SVD based
    initialization: A head start for nonnegative
    matrix factorization - Pattern Recognition, 2008

    http://tinyurl.com/nndsvd
    """
    check_non_negative(X, "NMF initialization")
    if variant not in (None, 'a', 'ar'):
        raise ValueError("Invalid variant name")

    U, S, V = randomized_svd(X, n_components)
    W, H = np.zeros(U.shape), np.zeros(V.shape)

    # The leading singular triplet is non-negative
    # so it can be used as is for initialization.
    W[:, 0] = np.sqrt(S[0]) * np.abs(U[:, 0])
    H[0, :] = np.sqrt(S[0]) * np.abs(V[0, :])

    for j in range(1, n_components):
        x, y = U[:, j], V[j, :]

        # extract positive and negative parts of column vectors
        x_p, y_p = np.maximum(x, 0), np.maximum(y, 0)
        x_n, y_n = np.abs(np.minimum(x, 0)), np.abs(np.minimum(y, 0))

        # and their norms
        x_p_nrm, y_p_nrm = LA.norm(x_p), LA.norm(y_p)
        x_n_nrm, y_n_nrm = LA.norm(x_n), LA.norm(y_n)

        m_p, m_n = x_p_nrm * y_p_nrm, x_n_nrm * y_n_nrm

        # choose update
        if m_p > m_n:
            u = x_p / x_p_nrm
            v = y_p / y_p_nrm
            sigma = m_p
        else:
            u = x_n / x_n_nrm
            v = y_n / y_n_nrm
            sigma = m_n

        lbd = np.sqrt(S[j] * sigma)
        W[:, j] = lbd * u
        H[j, :] = lbd * v

    W[W < eps] = 0
    H[H < eps] = 0

    if variant == "a":
        avg = X.mean()
        W[W == 0] = avg
        H[H == 0] = avg
    elif variant == "ar":
        random_state = check_random_state(random_state)
        avg = X.mean()
        W[W == 0] = abs(avg * random_state.randn(len(W[W == 0])) / 100)
        H[H == 0] = abs(avg * random_state.randn(len(H[H == 0])) / 100)

    return W, H

In [0]:
# Adapted version of the NMF function to integrate graph-regularization
#
# See:
# https://github.com/luispedro/milk/blob/master/milk/unsupervised/nnmf/lee_seung.py
# https://www.researchgate.net/profile/Zhigang_Luo/publication/258350768_Limited-memory_fast_gradient_descent_method_for_graph_regularized_nonnegative_matrix_factorization/links/0c9605282f7f611648000000.pdf
from sklearn.utils import check_array
import warnings

def GNMF(X,L,lambd=0,n_components=None,tol=1e-4,max_iter=100,verbose=False):      
        X = check_array(X)
        check_non_negative(X, "NMF.fit")
        n_samples, n_features = X.shape
  
        if not n_components:
            n_components = n_features
        else:
            n_components = n_components
  
        #W, H = NBS_init(X,n_components)
        W = np.random.normal(0,1,(n_samples,n_components))**2
        H = np.random.normal(0,1,(n_components,n_features))**2
        
        reconstruction_err_ = LA.norm(X - np.dot(W, H))
        eps=1e-4#spacing(1) #10e-14
        Lp = (abs(L)+L)/2
        Lm = (abs(L)-L)/2
       
        for n_iter in range(1, max_iter + 1):
            if verbose:
                print "Iteration =", n_iter,"/",max_iter, "— Error =", reconstruction_err_,"/",tol
            
            h1=lambd*np.dot(H,Lm)+np.dot(W.T,(X+eps)/(np.dot(W,H)+eps))
            h2=lambd*np.dot(H,Lp)+np.dot(W.T,np.ones(X.shape))
            H = np.multiply(H,(h1+eps)/(h2+eps))
            H[H<=0]=eps
            H[np.isnan(H)]=eps
            
            w1=np.dot((X+eps)/(np.dot(W,H)+eps),H.T)
            w2=np.dot(np.ones(X.shape),H.T)
            W = np.multiply(W,(w1+eps)/(w2+eps))
            W[H<=0]=eps
            W[np.isnan(W)]=eps            
            
            if not sp.issparse(X):
                if reconstruction_err_ > LA.norm(X - np.dot(W, H)):
                    H=(1-eps)*H+eps*np.random.normal(0,1,(n_components,n_features))**2
                    W=(1-eps)*W+eps*np.random.normal(0,1,(n_samples,n_components))**2
                reconstruction_err_ = LA.norm(X - np.dot(W, H))
            else:
                norm2X = np.sum(X.data ** 2)  # Ok because X is CSR
                normWHT = np.trace(np.dot(np.dot(H.T, np.dot(W.T, W)), H))
                cross_prod = np.trace(np.dot((X * H.T).T, W))
                reconstruction_err_ = sqrt(norm2X + normWHT - 2. * cross_prod)
                    
            if reconstruction_err_<tol:
                warnings.warn("Tolerance error reached during fit")
                break
            
            if np.isnan(W).any() or np.isnan(H).any():
                warnings.warn("NaN values at "+ str(n_iter)+" Error="+str(reconstruction_err_))
                break
                              
            if n_iter == max_iter:
                warnings.warn("Iteration limit reached during fit")
  
        return np.squeeze(np.asarray(W)), np.squeeze(np.asarray(H)), reconstruction_err_

In [0]:
gnmfFactor = 0.7
WNMF, stratipyCompG, reconstruction_err_ = GNMF(np.matrix(mutFinal),np.matrix(netFinal),0.,n_components=3,tol=1e-3)
WNMFDiff, stratipyCompGDiff, reconstruction_err_Diff = GNMF(np.matrix(mutDiffused),np.matrix(netFinal),0.,n_components=3,tol=1e-3)
WNMFQDiff, stratipyCompGQDiff, reconstruction_err_QDiff = GNMF(np.matrix(mutQDiffused),np.matrix(netFinal),0.,n_components=3,tol=1e-3)
WGNMF, stratipyCompGNMF, reconstruction_err_ = GNMF(np.matrix(mutFinal),np.matrix(netFinal),gnmfFactor,n_components=3,tol=1e-3)
WGNMFDiff, stratipyCompGNMFDiff, reconstruction_err_Diff = GNMF(np.matrix(mutDiffused),np.matrix(netFinal),gnmfFactor,n_components=3,tol=1e-3)
WGNMFQDiff, stratipyCompGNMFQDiff, reconstruction_err_QDiff = GNMF(np.matrix(mutDiffused),np.matrix(netFinal),gnmfFactor,n_components=3,tol=1e-3)

In [0]:
plt.figure(1,figsize=(20,10))
plt.subplot(611)
plt.plot(stratipyCompG.T/stratipyCompGNMF.max())
plt.ylabel("Weight")
plt.xlabel("Genes")
plt.title("NMF decomposition on raw mutation profiles")
plt.xlim([0,stratipyCompG.shape[1]])
plt.subplot(612)
plt.plot(stratipyCompGDiff.T/stratipyCompGNMFDiff.max())
plt.ylabel("Weight")
plt.xlabel("Genes")
plt.title("NMF decomposition on diffused mutation profiles")
plt.xlim([0,stratipyCompGDiff.shape[1]])
plt.subplot(613)
plt.plot(stratipyCompGQDiff.T/stratipyCompGQDiff.max())
plt.ylabel("Weight")
plt.xlabel("Genes")
plt.title("NMF decomposition on quantile normalized diffused mutation profiles")
plt.xlim([0,stratipyCompGDiff.shape[1]])
plt.subplot(614)
plt.plot(stratipyCompGNMF.T/stratipyCompG.max())
plt.ylabel("Weight")
plt.xlabel("Genes")
plt.title("GNMF decomposition on raw mutation profiles")
plt.xlim([0,stratipyCompGNMF.shape[1]])
plt.subplot(615)
plt.plot(stratipyCompGNMFDiff.T/stratipyCompGDiff.max())
plt.ylabel("Weight")
plt.xlabel("Genes")
plt.title("GNMF decomposition on diffused mutation profiles")
plt.xlim([0,stratipyCompGNMFDiff.shape[1]])
plt.subplot(616)
plt.plot(stratipyCompGNMFQDiff.T/stratipyCompGNMFQDiff.max())
plt.ylabel("Weight")
plt.xlabel("Genes")
plt.title("GNMF decomposition on quantile normalized diffused mutation profiles")
plt.xlim([0,stratipyCompGNMFQDiff.shape[1]])
plt.legend({'Component 1','Component 2','Component 3'})
plt.show()

In [0]:
Stratification=np.argmax(stratipyCompGQDiff,axis=0)
Weights=np.array([stratipyCompGQDiff[i,idx] for idx,i in enumerate(Stratification)])
plt.figure(1,figsize=(16,5))
plt.hist(Weights,200)
plt.show()

In [0]:
for comp in range(3):
    selectedGenes=symbols[((Stratification==comp)*(Weights>0.1))]
    print comp+1,len(selectedGenes)
    for g in selectedGenes:
        print g[0][0]
    print '\n'

In [0]:
print "Type of Cancers:"
for c in sorted(list(set(cancer))):
    print "- "+c.capitalize()+":"
    for p in range(3):
        print "Component",p,":", np.round(1000*float(len([v for i,v in enumerate(np.argmax(WNMFDiff,axis=1)==p) if v and cancer[i]==c]))/np.sum(np.array([cancer[i]==c for i in range(WNMFDiff.shape[0])])))/10

print "\nGrade of Cancers:"
for c in sorted(list(set(grade))):
    print "- "+c.capitalize()+":"
    for p in range(3):
        print "Component",p,":", np.round(1000*float(len([v for i,v in enumerate(np.argmax(WNMFDiff,axis=1)==p) if v and grade[i]==c]))/np.sum(np.array([grade[i]==c for i in range(WNMFDiff.shape[0])])))/10

# Consensus Clustering

In [0]:
patientsNum, genesNum = mutFinal.shape
permutationsNum = 1000
runBootstrap = False

if runBootstrap:
    genesClusteringNMF=np.zeros([genesNum,permutationsNum])*np.nan
    patientsClusteringNMF=np.zeros([patientsNum,permutationsNum])*np.nan
    genesClusteringNMFDiff=np.zeros([genesNum,permutationsNum])*np.nan
    patientsClusteringNMFDiff=np.zeros([patientsNum,permutationsNum])*np.nan
    genesClusteringNMFQDiff=np.zeros([genesNum,permutationsNum])*np.nan
    patientsClusteringNMFQDiff=np.zeros([patientsNum,permutationsNum])*np.nan
    genesClusteringGNMF=np.zeros([genesNum,permutationsNum])*np.nan
    patientsClusteringGNMF=np.zeros([patientsNum,permutationsNum])*np.nan
    genesClusteringGNMFDiff=np.zeros([genesNum,permutationsNum])*np.nan
    patientsClusteringGNMFDiff=np.zeros([patientsNum,permutationsNum])*np.nan
    genesClusteringGNMFQDiff=np.zeros([genesNum,permutationsNum])*np.nan
    patientsClusteringGNMFQDiff=np.zeros([patientsNum,permutationsNum])*np.nan
    for perm in range(permutationsNum):
        patientsSelected=np.random.permutation(patientsNum)[0:int(patientsNum*0.8)]
        genesSelected=np.random.permutation(genesNum)[0:int(genesNum*0.8)]
        subselectionFiltered=mutFinal[patientsSelected,:][:,genesSelected]
        subselectionDiffused=mutDiffused[patientsSelected,:][:,genesSelected]
        subselectionQDiffused=mutQDiffused[patientsSelected,:][:,genesSelected]
        subPPI=netFinal[genesSelected,:][:,genesSelected]
        WNMF, stratipyCompNMF, reconstruction_err_ = GNMF(np.matrix(subselectionFiltered),np.matrix(subPPI),0.,n_components=3,tol=1e-3)
        WNMFDiff, stratipyCompNMFDiff, reconstruction_err_Diff = GNMF(np.matrix(subselectionDiffused),np.matrix(subPPI),0.,n_components=3,tol=1e-3)
        WNMFQDiff, stratipyCompNMFQDiff, reconstruction_err_QDiff = GNMF(np.matrix(subselectionQDiffused),np.matrix(subPPI),0.,n_components=3,tol=1e-3)
        WGNMF, stratipyCompGNMF, reconstruction_err_ = GNMF(np.matrix(subselectionFiltered),np.matrix(subPPI),diffusionFactor,n_components=3,tol=1e-3)
        WGNMFDiff, stratipyCompGNMFDiff, reconstruction_err_Diff = GNMF(np.matrix(subselectionDiffused),np.matrix(subPPI),diffusionFactor,n_components=3,tol=1e-3)
        WGNMFQDiff, stratipyCompGNMFQDiff, reconstruction_err_QDiff = GNMF(np.matrix(subselectionQDiffused),np.matrix(subPPI),diffusionFactor,n_components=3,tol=1e-3)
        genesClusteringNMF[genesSelected,perm]=np.argmax(stratipyCompNMF,axis=0)
        genesClusteringNMFDiff[genesSelected,perm]=np.argmax(stratipyCompNMFDiff,axis=0)
        genesClusteringNMFQDiff[genesSelected,perm]=np.argmax(stratipyCompNMFQDiff,axis=0)
        genesClusteringGNMF[genesSelected,perm]=np.argmax(stratipyCompGNMF,axis=0)
        genesClusteringGNMFDiff[genesSelected,perm]=np.argmax(stratipyCompGNMFDiff,axis=0)
        genesClusteringGNMFQDiff[genesSelected,perm]=np.argmax(stratipyCompGNMFQDiff,axis=0)
        patientsClusteringNMF[patientsSelected,perm]=np.argmax(WNMF,axis=1)
        patientsClusteringNMFDiff[patientsSelected,perm]=np.argmax(WNMFDiff,axis=1)
        patientsClusteringNMFQDiff[patientsSelected,perm]=np.argmax(WNMFQDiff,axis=1)
        patientsClusteringGNMF[patientsSelected,perm]=np.argmax(WGNMF,axis=1)
        patientsClusteringGNMFDiff[patientsSelected,perm]=np.argmax(WGNMFDiff,axis=1)
        patientsClusteringGNMFQDiff[patientsSelected,perm]=np.argmax(WGNMFQDiff,axis=1)
    savemat(dataFolder+'bootstrap.mat',{'genesClusteringNMF': genesClusteringNMF,
                                        'patientsClusteringNMF': patientsClusteringNMF,
                                        'genesClusteringNMFDiff': genesClusteringNMFDiff,
                                        'patientsClusteringNMFDiff': patientsClusteringNMFDiff,
                                        'genesClusteringNMFQDiff': genesClusteringNMFQDiff,
                                        'patientsClusteringNMFQDiff': patientsClusteringNMFQDiff,
                                        'genesClusteringGNMF': genesClusteringGNMF,
                                        'patientsClusteringGNMF': patientsClusteringGNMF,
                                        'genesClusteringGNMFDiff': genesClusteringGNMFDiff,
                                        'patientsClusteringGNMFDiff': patientsClusteringGNMFDiff,
                                        'genesClusteringGNMFQDiff': genesClusteringGNMFQDiff,
                                        'patientsClusteringGNMFQDiff': patientsClusteringGNMFQDiff},
            do_compression=True)
else:
    bootstrap_data = loadmat(dataFolder+'bootstrap.mat')
    genesClusteringNMF=bootstrap_data['genesClusteringNMF']
    patientsClusteringNMF=bootstrap_data['patientsClusteringNMF']
    genesClusteringNMFDiff=bootstrap_data['genesClusteringNMFDiff']
    patientsClusteringNMFDiff=bootstrap_data['patientsClusteringNMFDiff']
    genesClusteringNMFQDiff=bootstrap_data['genesClusteringNMFQDiff']
    patientsClusteringNMFQDiff=bootstrap_data['patientsClusteringNMFQDiff']
    genesClusteringGNMF=bootstrap_data['genesClusteringGNMF']
    patientsClusteringGNMF=bootstrap_data['patientsClusteringGNMF']
    genesClusteringGNMFDiff=bootstrap_data['genesClusteringGNMFDiff']
    patientsClusteringGNMFDiff=bootstrap_data['patientsClusteringGNMFDiff']
    genesClusteringGNMFQDiff=bootstrap_data['genesClusteringGNMFQDiff']
    patientsClusteringGNMFQDiff=bootstrap_data['patientsClusteringGNMFQDiff']

In [0]:
import itertools
from scipy.spatial.distance import pdist

def ConcensusClustering(mat):
    distance = np.zeros([patientsClusteringGNMFDiff.shape[0], patientsClusteringGNMFDiff.shape[0]])
    for patient1 in range(mat.shape[0]):
        for patient2 in range(patient1+1,mat.shape[0]):
            I=(np.isnan(mat[[patient1,patient2],:]).sum(axis=0)==0).sum()
            M=(mat[patient1,:]==mat[patient2,:]).sum()
            distance[patient1,patient2]=float(M)/I
            distance[patient2,patient1]=float(M)/I
    return distance

def JaccardDistance(mat):
    return squareform(np.array([pdist(mat[s][:, ~np.isnan(mat[s]).any(axis=0)], "jaccard") for s in map(list, itertools.combinations(range(mat.shape[0]), 2))]).ravel())

In [0]:
dataNMF = patientsClusteringNMF
dataNMFDiff = patientsClusteringNMFDiff
dataNMFQDiff = patientsClusteringNMFQDiff
dataGNMF = patientsClusteringGNMF
dataGNMFDiff = patientsClusteringGNMFDiff
dataGNMFQDiff = patientsClusteringGNMFQDiff
dNMF = ConcensusClustering(dataNMF)
dNMFDiff = ConcensusClustering(dataNMFDiff)
dNMFQDiff = ConcensusClustering(dataNMFQDiff)
dGNMF = ConcensusClustering(dataGNMF)
dGNMFDiff = ConcensusClustering(dataGNMFDiff)
dGNMFQDiff = ConcensusClustering(dataGNMFQDiff)

In [0]:
plt.figure(1,figsize=(20,10))
plt.subplot(321)
plt.imshow((dNMF))
plt.title('NMF')
plt.subplot(322)
plt.imshow((dNMFDiff))
plt.title('NMF Diff')
plt.subplot(323)
plt.imshow((dGNMF))
plt.title('GNMF')
plt.subplot(324)
plt.imshow((dGNMFDiff))
plt.title('GNMF Diff')
plt.subplot(325)
plt.imshow((dNMFQDiff))
plt.title('NMF QDiff')
plt.subplot(326)
plt.imshow((dGNMFQDiff))
plt.title('GNMF QDiff')

In [0]:
from scipy.cluster.hierarchy import linkage
ZNMF = linkage(dNMF)
ZNMFDiff = linkage(dNMFDiff)
ZNMFQDiff = linkage(dNMFQDiff)
ZGNMF = linkage(dGNMF)
ZGNMFDiff = linkage(dGNMFDiff)
ZGNMFQDiff = linkage(dGNMFQDiff)
from scipy.cluster.hierarchy import fcluster
clNMF = fcluster(ZNMF,1)
clNMFDiff = fcluster(ZNMFDiff,1)
clNMFQDiff = fcluster(ZNMFQDiff,1)
clGNMF = fcluster(ZGNMF,1)
clGNMFDiff = fcluster(ZGNMFDiff,1)
clGNMFQDiff = fcluster(ZGNMFQDiff,1)

In [0]:
from scipy.cluster.hierarchy import dendrogram
plt.figure(1,figsize=(16,16))
plt.subplot(611)
a=dendrogram(ZNMF,count_sort='ascending');
idxNMF=np.array(a['leaves'])
plt.title('NMF')
plt.subplot(612)
a=dendrogram(ZNMFDiff,count_sort='ascending');
idxNMFDiff=np.array(a['leaves'])
plt.title('NMF Diff')
plt.subplot(613)
a=dendrogram(ZNMFQDiff,count_sort='ascending');
idxNMFQDiff=np.array(a['leaves'])
plt.title('NMF QDiff')
plt.subplot(614)
a=dendrogram(ZGNMF,count_sort='ascending');
idxGNMF=np.array(a['leaves'])
plt.title('GNMF')
plt.subplot(615)
a=dendrogram(ZGNMFDiff,count_sort='ascending');
idxGNMFDiff=np.array(a['leaves'])
plt.title('GNMF Diff')
plt.subplot(616)
a=dendrogram(ZGNMFQDiff,count_sort='ascending');
idxGNMFQDiff=np.array(a['leaves'])
plt.title('GNMF Diff')

In [0]:
for cl in set(a['color_list']):
    selectedPatients=[(i,e==cl) for i,e in enumerate(a['color_list'])]
    print cl, np.array([v for u,v in selectedPatients]).sum()
    print [grade[k] for k in [u for u,v in selectedPatients if v]]

In [0]:
plt.figure(1,figsize=(20,10))
plt.subplot(321)
plt.imshow(dNMF[idxNMF,:][:,idxNMF])
plt.title('NMF')
plt.subplot(322)
plt.imshow(dGNMF[idxGNMF,:][:,idxGNMF])
plt.title('GNMF')
plt.subplot(323)
plt.imshow(dNMFDiff[idxNMFDiff,:][:,idxNMFDiff])
plt.title('NMF Diff')
plt.subplot(324)
plt.imshow(dGNMFDiff[idxGNMFDiff,:][:,idxGNMFDiff])
plt.title('GNMF Diff')
plt.subplot(325)
plt.imshow(dNMFQDiff[idxNMFQDiff,:][:,idxNMFQDiff])
plt.title('NMF QDiff')
plt.subplot(326)
plt.imshow(dGNMFQDiff[idxGNMFQDiff,:][:,idxGNMFQDiff])
plt.title('GNMF QDiff')

## Network visualization

In [0]:
import pandas as pd
tmp=[k for i,k in enumerate(good+bad) if degree[i]>0]
selectedGenes=[k for i,k in enumerate(tmp) if notAlone[i]]

df0=pd.DataFrame({'EntrezId':[g[0] for g in somatic['gene_id_all'][selectedGenes]],'Genes':[g[0][0] for g in somatic['gene_id_symbol'][selectedGenes]]})
df1=pd.DataFrame({'StartiPyDiff_1':stratipyCompGDiff[0,:].T,'StartiPyDiff_2':stratipyCompGDiff[1,:].T,'StartiPyDiff_3':stratipyCompGDiff[2,:].T,'StratiPyDiff_W':stratipyCompGDiff.sum(axis=0).T,'StratiPyDiff_Comp':np.argmax(stratipyCompGDiff, axis=0).T})
df2=pd.DataFrame({'StartiPy_1':stratipyCompG[0,:].T,'StartiPy_2':stratipyCompG[1,:].T,'StartiPy_3':stratipyCompG[2,:].T,'StratiPy_W':stratipyCompG.sum(axis=0).T,'StratiPy_Comp':np.argmax(stratipyCompG, axis=0).T})
df3=pd.DataFrame({'NNF_1':sklearnComp[0,:].T,'NNF_2':sklearnComp[1,:].T,'NNF_3':sklearnComp[2,:].T,'NNF_W':sklearnComp.sum(axis=0).T,'NNF_Comp':np.argmax(sklearnComp, axis=0).T})
df4=pd.DataFrame({'NNFDiff_1':sklearnCompDiff[0,:].T,'NNFDiff_2':sklearnCompDiff[1,:].T,'NNFDiff_3':sklearnCompDiff[2,:].T,'NNFDiff_W':sklearnCompDiff.sum(axis=0).T,'NNFDiff_Comp':np.argmax(sklearnCompDiff, axis=0).T})
pd.concat([df0,df1,df2,df3,df4],axis=1).to_csv(dataFolder+'StratificationResults.csv')

In [0]:
import networkx as nx
H=nx.from_numpy_matrix(np.matrix(nnnetFiltered))
nx.write_edgelist(H, dataFolder+"Hofree-edgelist.csv")

In [0]:
plt.figure(1,figsize=(16,10))
pos=nx.graphviz_layout(H,prog="neato")
node_color=np.argmax(stratipyCompGDiff, axis=0)
nx.draw(H,pos,with_labels=False,node_size=50,node_color=node_color,cmap = plt.cm.Pastel1)
cut = 1.05
xmax= cut*max(xx for xx,yy in pos.values())
ymax= cut*max(yy for xx,yy in pos.values())
plt.xlim(0,xmax)
plt.ylim(0,ymax)
plt.show()

## Check the effects of the parameters

In [0]:
err=np.zeros((20,11))
for ncomp in range(20):
    for smooth in range(11):
        print "Ncomp=",ncomp+1," Smooth=",smooth/10.,
        WDiff2,stratipyCompGDiff2,error = GNMF(np.matrix(mutFinal),np.matrix(netFinal),smooth/10.,n_components=ncomp+1,tol=1e-3,max_iter=5)
        err[ncomp,smooth]=error
        print " Error=",error

In [0]:
err2=np.zeros((20,11))
for ncomp in range(20):
    for smooth in range(11):
        print "Ncomp=",ncomp+1," Smooth=",smooth/10.,
        WDiff2,stratipyCompGDiff2,error = GNMF(np.matrix(mutQDiffused),np.matrix(netFinal),smooth/10.,n_components=ncomp+1,tol=1e-3,max_iter=5)
        err2[ncomp,smooth]=error
        print " Error=",error

In [0]:
plt.figure(figsize=(20,11))
plt.subplot(121)
plt.imshow(err, interpolation="nearest")
plt.gca().invert_yaxis()
plt.xticks(np.arange(11),np.arange(11)/10.)
plt.yticks(np.arange(20),np.arange(20)+1)
plt.ylabel("Number of Component(s)")
plt.xlabel("Smoothing factor")
plt.title("Absolute error")
plt.colorbar()
plt.subplot(122)
plt.imshow(err-np.matrix(np.mean(err,axis=1)).T*np.matrix(np.ones(11)), interpolation="nearest")
plt.gca().invert_yaxis()
plt.xticks(np.arange(11),np.arange(11)/10.)
plt.yticks(np.arange(20),np.arange(20)+1)
plt.ylabel("Number of Component(s)")
plt.xlabel("Smoothing factor")
plt.title("Relative error by number of component")
plt.colorbar()
plt.show()

In [0]:
plt.figure(figsize=(20,10))
plt.subplot(121)
plt.imshow(err2, interpolation="nearest")
plt.gca().invert_yaxis()
plt.xticks(np.arange(11),np.arange(11)/10.)
plt.yticks(np.arange(20),np.arange(20)+1)
plt.ylabel("Number of Component(s)")
plt.xlabel("Smoothing factor")
plt.title("Absolute error")
plt.colorbar()
plt.subplot(122)
plt.imshow(err2-np.matrix(np.mean(err2,axis=1)).T*np.matrix(np.ones(11)), interpolation="nearest")
plt.gca().invert_yaxis()
plt.xticks(np.arange(11),np.arange(11)/10.)
plt.yticks(np.arange(20),np.arange(20)+1)
plt.ylabel("Number of Component(s)")
plt.xlabel("Smoothing factor")
plt.title("Relative error by number of component")
plt.colorbar()
plt.show()

In [0]:
plt.figure(figsize=(16,10))
plt.subplot(121)
plt.plot(np.vstack((err.mean(axis=1)-err.mean(),err2.mean(axis=1)-err2.mean())).T)
plt.ylabel("Average relative reconstruction error")
plt.xlabel("Number of Component(s)")
plt.subplot(122)
plt.plot(np.vstack((err.mean(axis=0)-err.mean(),err2.mean(axis=0)-err2.mean())).T)
plt.ylabel("Average relative reconstruction error")
plt.xlabel("Smoothing factor")
plt.legend({"Diffused","Filtered"})
plt.show()

## Under construction ...